# **TEXT AUTO ENCODER USING SUPERVISED EFOREST ON IMDB TEXT DATASET WITH 500 TREE**

In [ ]:
from keras.datasets import imdb
import numpy as np


In [ ]:
max_features= 100 # NUMBER OF WORD OR VOCABULARY SIZE
(X_tr, y_tr), (X_te, y_te) =  imdb.load_data(num_words=max_features)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
X_train = X_tr[:500]
y_train = y_tr[:500]
X_test = X_te[:500]
y_test = y_te[:500]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [ ]:
print("Categories:", np.unique(y_train))
print("Number of unique words:", len(np.unique(np.hstack(X_train))))

Categories: [0 1]
Number of unique words: 98


In [ ]:
length = [len(i) for i in X_train]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 257.8
Standard Deviation: 196


In [ ]:
print("Label:", y_train[0])
print(X_train[0])

Label: 1
[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 4, 2, 36, 2, 5, 25, 2, 43, 2, 2, 50, 2, 2, 9, 35, 2, 2, 5, 2, 4, 2, 2, 2, 2, 2, 2, 39, 4, 2, 2, 2, 17, 2, 38, 13, 2, 4, 2, 50, 16, 6, 2, 2, 19, 14, 22, 4, 2, 2, 2, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 2, 12, 8, 2, 8, 2, 5, 4, 2, 2, 16, 2, 66, 2, 33, 4, 2, 12, 16, 38, 2, 5, 25, 2, 51, 36, 2, 48, 25, 2, 33, 6, 22, 12, 2, 28, 77, 52, 5, 14, 2, 16, 82, 2, 8, 4, 2, 2, 2, 15, 2, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 7, 4, 2, 2, 13, 2, 88, 4, 2, 15, 2, 98, 32, 2, 56, 26, 2, 6, 2, 2, 18, 4, 2, 22, 21, 2, 2, 26, 2, 5, 2, 30, 2, 18, 51, 36, 28, 2, 92, 25, 2, 4, 2, 65, 16, 38, 2, 88, 12, 16, 2, 5, 16, 2, 2, 2, 32, 15, 16, 2, 19, 2, 32]


##**Pre-processing**

In [ ]:
# Used for pre-processing data
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocessing(text):
    #splits the word and checks each character to see if it contains any standard punctuations, 
    #if so it will be replaced with a blank or else it just don’t replace with blank
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())
    
    #tokenizes the sentences into words based on whitespaces and puts them together as a list for applying further steps
    tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
    
    #Converting all the cases (upper, lower and proper) into lower case reduces duplicates in corpus
    tokens = [word.lower() for word in tokens]
    
    #Stop words are the words that do not carry much of weight in understanding the sentence; 
    #they are used for connecting words and so on. We have removed them with the following line of code
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    
    #Stemming applied on the words using Porter stemmer which stems the extra suffixes from the words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    #POS tagging is a prerequisite for lemmatization, based on whether the word 
    #is noun or verb or and so on. it will reduce it to the root word
    tagged_corpus = pos_tag(tokens)

    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']
    lemmatizer = WordNetLemmatizer()

    #  If the tag for any word falls under the respective noun or verb tags category, 
    #  n or v will be applied accordingly in lemmatize function
    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')

        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    
    #we need to join it back to form stings and the following function performs the same
    pre_proc_text =   " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])

    return pre_proc_text

###**PREPROCESSING ON TRAIN AND TEST DATA**

In [ ]:
# A dictionary mapping words to an integer index
vocabulary = imdb.get_word_index()

index = dict([(value, key) for (key, value) in vocabulary.items()])

In [ ]:
'''converts encoded text to human readable form.
    each integer in the text is looked up in the index, and 
    replaced by the corresponding word.
'''
X_train = [' '.join([index.get(i, '?') for i in x]) for x in X_train]
X_test = [' '.join([index.get(i, '?') for i in x]) for x in X_test]

In [ ]:
X_train_preprocessed = []
for i in X_train:
    X_train_preprocessed.append(preprocessing(i))

X_test_preprocessed = []
for i in X_test:
    X_test_preprocessed.append(preprocessing(i))
    

In [ ]:
X_train_preprocessed[0]

'movi film get movi movi stori one br br one film'

In [ ]:
X_test_preprocessed[0]

'br one'

### **BUILDING TF-IDF vectorizer**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from keras.preprocessing.sequence import pad_sequences

vectorizer = TfidfVectorizer(max_features= max_features)
vectorizer.fit(X_train_preprocessed) 

X_train_2 = vectorizer.transform(X_train_preprocessed).todense()

X_test_2 = vectorizer.transform(X_test_preprocessed).todense()

In [ ]:
np.set_printoptions(threshold=np.inf) # not usefull for this code just to prevent truncation of output result
print(X_train_2[0][:20])

[[0.         0.         0.30788979 0.         0.         0.40583095
  0.         0.31603217 0.         0.         0.         0.
  0.         0.57512642 0.         0.46078001 0.         0.
  0.         0.31231423 0.         0.         0.         0.        ]]


### **DEFINE NODE RULE**

In [ ]:
import copy
import numpy as np

In [ ]:
class NodeRule:

    LT = 0 # 'less than'
    LE = 1 # 'less or equal to'

    def __init__(self, attribute, threshold, op):
        # [lower_bound] [op_left] value [op_right] [upper_bound]
        assert(op in ['gt', 'le'])
        self.attribute = attribute
        if op == 'le':
            self.lower_bound = -np.inf
            self.upper_bound = threshold
            self.op_left = NodeRule.LT
            self.op_right = NodeRule.LE
        else:
            self.lower_bound = threshold
            self.upper_bound = np.inf
            self.op_left = NodeRule.LT
            self.op_right = NodeRule.LT
    
    def __and__(self, other):
        new_rule = copy.copy(self)
        if self.lower_bound > other.lower_bound:
            new_rule.lower_bound = self.lower_bound
            new_rule.op_left = self.op_left
        elif self.lower_bound < other.lower_bound:
            new_rule.lower_bound = other.lower_bound
            new_rule.op_left = other.op_left
        else:
            new_rule.lower_bound = self.lower_bound
            if NodeRule.LT in [self.op_left, other.op_left]:
                new_rule.op_left = NodeRule.LT
            else:
                new_rule.op_left = NodeRule.LE
        if self.upper_bound < other.upper_bound:
            new_rule.upper_bound = self.upper_bound
            new_rule.op_right = self.op_right
        elif self.upper_bound > other.upper_bound:
            new_rule.upper_bound = other.upper_bound
            new_rule.op_right = other.op_right
        else:
            new_rule.upper_bound = self.upper_bound
            if NodeRule.LT in [self.op_right, other.op_right]:
                new_rule.op_right = NodeRule.LT
            else:
                new_rule.op_right = NodeRule.LE
        return new_rule

    def __str__(self):
        s = str(self.lower_bound) + ' '
        s += '<=' if self.op_left == NodeRule.LE else '<'
        s += ' x_%i ' % self.attribute
        s += '<=' if self.op_right == NodeRule.LE else '<'
        s += ' ' + str(self.upper_bound)
        return s
    
    def __repr__(self):
        return self.__str__()

### **DEFINE PATH RULE**

In [ ]:
class PathRule:
    
    def __init__(self, n_attributes):
        self.n_attributes = n_attributes
        self.node_rules = [NodeRule(i, -np.inf, 'gt') for i in range(n_attributes)]
    
    def set_global_bounds(self, lower_bounds, upper_bounds):
        for i in range(self.n_attributes):
            node_rule = NodeRule(i, -np.inf, 'gt')
            node_rule.op_left = NodeRule.LE
            node_rule.op_right = NodeRule.LE
            node_rule.lower_bound = lower_bounds[i]
            node_rule.upper_bound = upper_bounds[i]
            self.add(node_rule)
    
    def add(self, node_rule):
        assert(isinstance(node_rule, NodeRule))
        j = node_rule.attribute
        self.node_rules[j] = self.node_rules[j].__and__(node_rule)
    
    def sample(self, sampling='mean'):
        sample = np.empty(self.n_attributes)
        for i, rule in enumerate(self):
            if sampling == 'mean':
                sample[i] = (rule.lower_bound + rule.upper_bound) / 2.
            else:
                sample[i] = np.random.uniform(rule.lower_bound, rule.upper_bound)
        return sample
    
    def compute_volume(self):
        log_volume = 1
        for i in range(self.n_attributes):
            node_rule = self.node_rules[i]
            log_volume += (node_rule.upper_bound - node_rule.lower_bound)
            assert(node_rule.upper_bound >= node_rule.lower_bound)
        return log_volume
    
    def __len__(self):
        return len(self.node_rules)
    
    def __iter__(self):
        self.current_attribute = 0
        return self
    
    def __next__(self):
        self.current_attribute += 1
        try:
            result = self.node_rules[self.current_attribute-1]
        except IndexError:
            raise StopIteration
        return result
    
    def __str__(self):
        return str(self.node_rules)
    
    def __repr__(self):
        return self.__str__()

## **RANDOME TREE MODEL**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeRegressor

In [ ]:
class CompletelyRandomTree:

    def __init__(self, sklearn_extra_tree, n_attributes):
        self.n_attributes = n_attributes
        self.tree = sklearn_extra_tree
        self.n_nodes = self.tree.tree_.node_count
        self.parent_ids = np.full(self.n_nodes, -1, dtype=np.int)
        self.operators = [None] * self.n_nodes
        tree = self.tree.tree_
        for i in range(self.n_nodes):
            parent_from_left = np.where(tree.children_left == i)[0]
            parent_from_right = np.where(tree.children_right == i)[0]
            if len(parent_from_left) == 1:
                self.parent_ids[i] = parent_from_left[0]
                self.operators[i] = 'le'
            elif len(parent_from_right) == 1:
                self.parent_ids[i] = parent_from_right[0]
                self.operators[i] = 'gt'
        self.leaf_to_node = np.where(tree.feature == -2)[0]
        self.node_to_leaf = {node_id: leaf_id for leaf_id, node_id in enumerate(self.leaf_to_node)}
        self.n_leafs = len(self.leaf_to_node)
    
    def encode(self, X):
        node_ids = self.tree.apply(X)
        return [self.node_to_leaf[node_id] for node_id in node_ids]
    
    def get_path_rule(self, default_path_rule, leaf_id):
        path_rule = copy.deepcopy(default_path_rule)
        tree = self.tree.tree_
        current_id = self.leaf_to_node[leaf_id]
        while current_id != 0:
            attribute = tree.feature[self.parent_ids[current_id]]
            bound = tree.threshold[self.parent_ids[current_id]]
            op = self.operators[current_id]
            path_rule.add(NodeRule(attribute, bound, op))
            current_id = self.parent_ids[current_id]
        return path_rule

## **DEFINE ENCODER FOREST**

In [ ]:
class EncoderForest:

    def __init__(self, n_components):
        self.trees = list()
        self.n_attributes = 0
        self.in_size = 0
        self.out_size = n_components
        self.global_lower_bounds = None
        self.global_upper_bounds = None
        self.default_path_rule = None
        self.unsupervised = True
    
    def fit(self, X, y=None, max_depth=10):
        self.unsupervised = (y is None)
        self.n_attributes = X.shape[1]
        self.in_size = X.shape[0]
        if y is None:
            forest = RandomTreesEmbedding(self.out_size, max_depth=max_depth)
            forest.fit(X)
        else:
            forest = RandomForestClassifier(n_estimators=self.out_size, max_depth=max_depth)
            forest.fit(X, y)
        for i in range(self.out_size):
            self.trees.append(CompletelyRandomTree(forest.estimators_[i], self.n_attributes))
        
        self.global_lower_bounds = np.min(X, axis=0).astype(np.double)
        self.global_upper_bounds = np.max(X, axis=0).astype(np.double)

        self.default_path_rule = PathRule(self.n_attributes)
        self.default_path_rule.set_global_bounds(self.global_lower_bounds, self.global_upper_bounds)
    
    def encode(self, X):
        n_samples = X.shape[0]
        out_dim = len(self.trees)
        encoded = np.empty((n_samples, out_dim), dtype=np.int)
        for i in range(out_dim):
            encoded[:, i] = self.trees[i].encode(X)
        return encoded

    def compute_rule_list(self, x):
        out_dim = len(self.trees)
        rule_list = list()
        for i in range(out_dim):
            path_rule = self.trees[i].get_path_rule(self.default_path_rule, x[i])
            rule_list.append(path_rule)
        return rule_list
    
    def decode(self, x):
        rule_list = self.compute_rule_list(x)
        MCR = self.calculate_MCR(rule_list)
        return MCR.sample(sampling='mean')

    def calculate_MCR(self, path_rule_list):
        MCR = copy.deepcopy(self.default_path_rule)
        n_attributes = len(MCR)
        n_trees = len(path_rule_list)

        for i in range(len(path_rule_list)):
            path_rule = path_rule_list[i]
            print("Volume of path rule %i: %f" % (i, path_rule.compute_volume()))
            for node_rule in path_rule.node_rules:
                MCR.add(node_rule)
        print("Volume of MCR: %f" % MCR.compute_volume())
        return MCR

In [ ]:
X_train = np.array(X_train_2)
X_test = np.array(X_test_2)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))
y_train = y_train.reshape((y_train.shape[0], -1))
y_test = y_test.reshape((y_test.shape[0], -1))
print("[IMDB] x_train.shape={}, y_train.shape={}".format(X_train.shape, y_train.shape))
print("[IMDB] x_test.shape={}, y_test.shape={}".format(X_test.shape, y_test.shape))

[IMDB] x_train.shape=(500, 24), y_train.shape=(500, 1)
[IMDB] x_test.shape=(500, 24), y_test.shape=(500, 1)


In [ ]:
print("Start Autoencoder using supervised model")
#eforest = RandomTreesEmbedding(n_estimators=n_trees, max_depth=None, n_jobs=-1, random_state=0)
encoder = EncoderForest(500)
encoder.fit(X_train, y_train, max_depth=5)
print("end fit on supervised model")

Start Autoencoder using supervised model


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


end fit on supervised model


### **MODEL EVALUATION**

In [ ]:
X_encode = encoder.encode(X_test)     

In [ ]:
X_encode.shape

(500, 500)

In [ ]:
results = []
for j in range(len(X_encode)):
    results.append(encoder.decode(X_encode[j]))

Streaming output truncated to the last 5000 lines.
Volume of path rule 10: 17.686466
Volume of path rule 11: 19.115204
Volume of path rule 12: 18.081898
Volume of path rule 13: 16.917600
Volume of path rule 14: 17.619666
Volume of path rule 15: 17.139917
Volume of path rule 16: 18.248650
Volume of path rule 17: 18.323529
Volume of path rule 18: 18.120534
Volume of path rule 19: 17.462237
Volume of path rule 20: 18.513810
Volume of path rule 21: 17.507701
Volume of path rule 22: 17.431191
Volume of path rule 23: 19.292527
Volume of path rule 24: 19.377904
Volume of path rule 25: 18.953957
Volume of path rule 26: 17.355616
Volume of path rule 27: 17.680770
Volume of path rule 28: 18.493584
Volume of path rule 29: 17.728653
Volume of path rule 30: 18.079054
Volume of path rule 31: 17.872377
Volume of path rule 32: 17.940011
Volume of path rule 33: 18.519758
Volume of path rule 34: 17.411236
Volume of path rule 35: 17.367896
Volume of path rule 36: 17.309824
Volume of path rule 37: 18.3756

In [ ]:
y_pred = np.array(results)

In [ ]:
y_pred.shape

(500, 24)

###**COSINE DISTANCE**

In [ ]:
X_test.shape

(500, 24)

In [ ]:
yt = X_test.flatten()
yt.shape

(12000,)

In [ ]:
yp = y_pred.flatten()
yp.shape

(12000,)

In [ ]:
from scipy.spatial.distance import cosine

cos_distance = cosine(yt, yp)

In [ ]:
cos_distance

0.004800696398161164